`pd.json_normalize` is a very useful way to unnest/ explode data structure that is commonly seen in JSON files.  
As long as the data/ a column with a dataframe contains a `dict`, we can use this method on it: 

In [3]:
import pandas as pd 
from IPython.display import display

data = {
    'name': 'team 1', 
    'scores': [10, 15, 220], 
}
display(pd.DataFrame(data).head())

print(f'score is type of {type(data["scores"])}')

pd.json_normalize(data=data, 
                  record_path='scores', 
                  record_prefix='scores.',
                  meta=['name'])

,name,scores
0,team 1,10
1,team 1,15
2,team 1,220


score is type of <class 'list'>


,scores.0,name
0,10,team 1
1,15,team 1
2,220,team 1


In [2]:
# use it on dict()
data = {
    'name': 'team 1', 
    'scores': [10, 15, 220], 
    'members': [
        {'name': 'Tom', 'skill_level': 10}, 
        {'name': 'Sam', 'skill_level': 8}, 
        {'name': 'Ken', 'skill_level': 6}
    ]
}
display(pd.DataFrame(data).head())

print(f'score is type of {type(data["members"])}')
pd.json_normalize(data=data, 
                  record_path=['members'], 
                  record_prefix='members.', 
                  meta=['name',])

,name,scores,members
0,team 1,10,"{'name': 'Tom', 'skill_level': 10}"
1,team 1,15,"{'name': 'Sam', 'skill_level': 8}"
2,team 1,220,"{'name': 'Ken', 'skill_level': 6}"


score is type of <class 'list'>


,members.name,members.skill_level,name
0,Tom,10,team 1
1,Sam,8,team 1
2,Ken,6,team 1


In [3]:
# even on more complex datatypes 
complex_data = [
    {
        "state": "Florida",
        "shortname": "FL",
        "info": {"governor": "Rick Scott"},
        "counties": [
            {"name": "Dade", "population": 12345},
            {"name": "Broward", "population": 40000},
            {"name": "Palm Beach", "population": 60000},
        ],
    },
    {
        "state": "Ohio",
        "shortname": "OH",
        "info": {"governor": "John Kasich"},
        "counties": [
            {"name": "Summit", "population": 1234},
            {"name": "Cuyahoga", "population": 1337},
        ],
    },
]

display(pd.DataFrame(complex_data).head())
pd.json_normalize(data=complex_data, 
                  record_path='counties', 
                  meta=['state', 'shortname', ['info', 'governor']])

,state,shortname,info,counties
0,Florida,FL,{'governor': 'Rick Scott'},"[{'name': 'Dade', 'population': 12345}, {'name..."
1,Ohio,OH,{'governor': 'John Kasich'},"[{'name': 'Summit', 'population': 1234}, {'nam..."


,name,population,state,shortname,info.governor
0,Dade,12345,Florida,FL,Rick Scott
1,Broward,40000,Florida,FL,Rick Scott
2,Palm Beach,60000,Florida,FL,Rick Scott
3,Summit,1234,Ohio,OH,John Kasich
4,Cuyahoga,1337,Ohio,OH,John Kasich


In [6]:
# However, we can see that a more even complex type of JSON files, 
# would eventually fails, 
# At the same time, hardcoding each columns for extract become more 
# cumbersome
even_more_complex_data = [
    {
        "city": "Greatpole",
        "info": {"ruler": "Tommy the Great"},
        "towns": [
            {"name": "Small Villa",
             "streets": [{"name": "street-1"}]},
            {"name": "Mid Villa",
             "streets": [{"name": "street-2"}]},
            {"name": "Large Villa",
             "streets": [{"name": "street-3"}]},
        ],
        "population": [
            {'year': '2022', 'size': 2000}, 
            {'year': '2021', 'size': 1000}
        ],
    }
]

display(pd.DataFrame(even_more_complex_data).head())
pd.json_normalize(data=even_more_complex_data, 
                  record_path=['towns'], 
                  record_prefix='towns.', 
                  meta=['city', ['info', 'ruler']])

,city,info,towns,population
0,Greatpole,{'ruler': 'Tommy the Great'},"[{'name': 'Small Villa', 'streets': [{'name': ...","[{'year': '2022', 'size': 2000}, {'year': '202..."


,towns.name,towns.streets,city,info.ruler
0,Small Villa,[{'name': 'street-1'}],Greatpole,Tommy the Great
1,Mid Villa,[{'name': 'street-2'}],Greatpole,Tommy the Great
2,Large Villa,[{'name': 'street-3'}],Greatpole,Tommy the Great


## Generalise approach

1. scan all the columns, explode columns first



In [4]:
from collections import namedtuple
from functools import reduce
import numpy as np

ColumnDtype = namedtuple('ColumnDtype', 'column dtype')

### Single step include all these functions
def get_dtypes(dataframe) -> tuple[str, type]:
    """Infer the dtypes of each column"""
    infer_row = dataframe.head(1)
    return [ColumnDtype(col, type(infer_row[col][0])) for col in infer_row]    


def select_inferred_dtypes(dataframe: pd.DataFrame, include: list=None):
    inferred_dtypes = get_dtypes(dataframe)
    
    if include:
        inferred_dtypes = filter(lambda info: info.dtype in include, inferred_dtypes)
        
    return list(map(lambda info: info.column, inferred_dtypes))


def explode_columns(dataframe: pd.DataFrame):
    """Explode array type columns iteratively"""
    dataframe = dataframe.copy()
    array_columns = select_inferred_dtypes(dataframe, include=[list, np.array])
    df = reduce(lambda df, col: df.explode(col), array_columns, dataframe)
    return df.reset_index(drop=True)


def normalise_json_columns(dataframe: pd.DataFrame):
    """Normalise (unnest) dict columns iteratively"""
    df = dataframe.copy()
    
    dict_columns = select_inferred_dtypes(df, include=[dict])
    
    for norm_col in dict_columns:
        all_columns = df.columns.tolist()   
        all_columns.remove(norm_col)
        
        norm_df = pd.json_normalize(df[norm_col], max_level=1)
        norm_df = norm_df.rename(columns={col: f'{norm_col}.{col}' for col in norm_df.columns})
        df = pd.concat([df[all_columns], norm_df], axis=1)
    return df 


In [16]:
# test for 1 iteration 
df = pd.DataFrame(even_more_complex_data)
df = explode_columns(df)
normalise_json_columns(df)

,city,info.ruler,towns.name,towns.streets,population.year,population.size
0,Greatpole,Tommy the Great,Small Villa,[{'name': 'street-1'}],2022,2000
1,Greatpole,Tommy the Great,Small Villa,[{'name': 'street-1'}],2021,1000
2,Greatpole,Tommy the Great,Mid Villa,[{'name': 'street-2'}],2022,2000
3,Greatpole,Tommy the Great,Mid Villa,[{'name': 'street-2'}],2021,1000
4,Greatpole,Tommy the Great,Large Villa,[{'name': 'street-3'}],2022,2000
5,Greatpole,Tommy the Great,Large Villa,[{'name': 'street-3'}],2021,1000


In [20]:
def normalise_json_dataframe(dataframe: pd.DataFrame):
    df = dataframe.copy()
    
    while True:
        df = explode_columns(df)
        df = normalise_json_columns(df)
        
        if not select_inferred_dtypes(df, include=[list, dict]): 
            break 
        
    return df

In [21]:
df = pd.DataFrame(even_more_complex_data)
normalise_json_dataframe(df)

,city,info.ruler,towns.name,population.year,population.size,towns.streets.name
0,Greatpole,Tommy the Great,Small Villa,2022,2000,street-1
1,Greatpole,Tommy the Great,Small Villa,2021,1000,street-1
2,Greatpole,Tommy the Great,Mid Villa,2022,2000,street-2
3,Greatpole,Tommy the Great,Mid Villa,2021,1000,street-2
4,Greatpole,Tommy the Great,Large Villa,2022,2000,street-3
5,Greatpole,Tommy the Great,Large Villa,2021,1000,street-3
